In [3]:
import pandas as pd

# Load your dataset
df = pd.read_csv('/Users/vaish2205/Desktop/BasicCompanyDataAsOneFile-2025-06-01.csv')

# Function to explore the dataset
def explore_dataset(df):
    print(" Basic Dataset Information\n" + "-"*40)
    
    print(f"Shape of dataset: {df.shape[0]} rows, {df.shape[1]} columns\n")
    
    print(" Column Names:")
    print(df.columns.tolist(), "\n")
    
    print(" Data Types:")
    print(df.dtypes, "\n")
    
    print(" First 5 Rows:")
    print(df.head(), "\n")
    
    print(" Last 5 Rows:")
    print(df.tail(), "\n")
    
    print(" Summary Statistics:")
    print(df.describe(include='all'), "\n")
    
    print(" Missing Values:")
    print(df.isnull().sum(), "\n")
    
    print(" Duplicate Rows Count:")
    print(df.duplicated().sum(), "\n")
    
    print(" Dataset Info:")
    df.info()

# Call the function to explore your loaded dataset
explore_dataset(df)


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/1836870682.py:4: DtypeWarning: Columns (3,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/vaish2205/Desktop/BasicCompanyDataAsOneFile-2025-06-01.csv')


 Basic Dataset Information
----------------------------------------
Shape of dataset: 5656584 rows, 55 columns

 Column Names:
['CompanyName', ' CompanyNumber', 'RegAddress.CareOf', 'RegAddress.POBox', 'RegAddress.AddressLine1', ' RegAddress.AddressLine2', 'RegAddress.PostTown', 'RegAddress.County', 'RegAddress.Country', 'RegAddress.PostCode', 'CompanyCategory', 'CompanyStatus', 'CountryOfOrigin', 'DissolutionDate', 'IncorporationDate', 'Accounts.AccountRefDay', 'Accounts.AccountRefMonth', 'Accounts.NextDueDate', 'Accounts.LastMadeUpDate', 'Accounts.AccountCategory', 'Returns.NextDueDate', 'Returns.LastMadeUpDate', 'Mortgages.NumMortCharges', 'Mortgages.NumMortOutstanding', 'Mortgages.NumMortPartSatisfied', 'Mortgages.NumMortSatisfied', 'SICCode.SicText_1', 'SICCode.SicText_2', 'SICCode.SicText_3', 'SICCode.SicText_4', 'LimitedPartnerships.NumGenPartners', 'LimitedPartnerships.NumLimPartners', 'URI', 'PreviousName_1.CONDATE', ' PreviousName_1.CompanyName', ' PreviousName_2.CONDATE', ' 

In [5]:
# Calculate percentage of missing values per column
missing_percent = df.isnull().mean() * 100

# Sort descending and print
missing_percent = missing_percent.sort_values(ascending=False)

print(missing_percent)


DissolutionDate                       100.000000
 PreviousName_10.CompanyName           99.999328
PreviousName_10.CONDATE                99.999328
 PreviousName_9.CompanyName            99.998922
PreviousName_9.CONDATE                 99.998922
 PreviousName_8.CompanyName            99.998197
PreviousName_8.CONDATE                 99.998197
PreviousName_7.CONDATE                 99.996676
 PreviousName_7.CompanyName            99.996676
PreviousName_6.CONDATE                 99.991921
 PreviousName_6.CompanyName            99.991921
 PreviousName_5.CompanyName            99.976187
PreviousName_5.CONDATE                 99.976187
PreviousName_4.CONDATE                 99.918131
 PreviousName_4.CompanyName            99.918131
PreviousName_3.CONDATE                 99.666389
 PreviousName_3.CompanyName            99.666389
RegAddress.CareOf                      99.463422
RegAddress.POBox                       99.380103
 PreviousName_2.CompanyName            98.411586
 PreviousName_2.COND

In [9]:
import pandas as pd

# Step 1: Clean column names
df.columns = df.columns.str.strip()

# Step 2: Drop columns with >80% missing values
threshold = 0.8
missing_fraction = df.isnull().mean()
cols_to_drop = missing_fraction[missing_fraction > threshold].index
df_clean = df.drop(columns=cols_to_drop)
print(f"🗑️ Dropped columns with >{threshold*100}% missing values:\n", list(cols_to_drop))

# Step 3: Drop known unnecessary columns (if still present)
if 'Returns.LastMadeUpDate' in df_clean.columns:
    df_clean.drop(columns=['Returns.LastMadeUpDate'], inplace=True)
    print("🗑️ Dropped 'Returns.LastMadeUpDate' due to >80% missing values.")

# Step 4: Fill address-related missing values with 'Unknown'
address_cols = [
    'RegAddress.AddressLine2',
    'RegAddress.County',
    'RegAddress.Country',
    'RegAddress.AddressLine1',
    'RegAddress.PostTown',
    'RegAddress.PostCode'
]
for col in address_cols:
    if col in df_clean.columns:
        df_clean[col].fillna('Unknown', inplace=True)

# Step 5: Convert date columns to datetime
date_cols = [
    'Returns.NextDueDate',
    'IncorporationDate',
    'Accounts.LastMadeUpDate',
    'ConfStmtLastMadeUpDate',
    'Accounts.NextDueDate',
    'ConfStmtNextDueDate'
]
for col in date_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Step 6: Fill mode for categorical numeric fields
for col in ['Accounts.AccountRefDay', 'Accounts.AccountRefMonth']:
    if col in df_clean.columns:
        mode_val = df_clean[col].mode(dropna=True)
        if not mode_val.empty:
            df_clean[col].fillna(mode_val[0], inplace=True)

# Step 7: Fill numeric missing values with median
numeric_cols = df_clean.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)

# Step 8: Fill remaining specific date columns with median date
for col in ['ConfStmtLastMadeUpDate']:
    if col in df_clean.columns:
        median_date = df_clean[col].median()
        df_clean[col].fillna(median_date, inplace=True)

# Step 9: Drop rows with missing values in key business columns
cols_to_check = [
    'Returns.NextDueDate',
    'IncorporationDate',
    'RegAddress.AddressLine2',
    'Accounts.LastMadeUpDate',
    'ConfStmtLastMadeUpDate',
    'Accounts.NextDueDate',
    'ConfStmtNextDueDate'
]
cols_to_check = [col for col in cols_to_check if col in df_clean.columns]
df_clean.dropna(subset=cols_to_check, inplace=True)
print(f"🧹 Dropped rows with missing values in critical columns: {cols_to_check}")


🗑️ Dropped columns with >80.0% missing values:
 ['RegAddress.CareOf', 'RegAddress.POBox', 'DissolutionDate', 'SICCode.SicText_2', 'SICCode.SicText_3', 'SICCode.SicText_4', 'PreviousName_1.CONDATE', 'PreviousName_1.CompanyName', 'PreviousName_2.CONDATE', 'PreviousName_2.CompanyName', 'PreviousName_3.CONDATE', 'PreviousName_3.CompanyName', 'PreviousName_4.CONDATE', 'PreviousName_4.CompanyName', 'PreviousName_5.CONDATE', 'PreviousName_5.CompanyName', 'PreviousName_6.CONDATE', 'PreviousName_6.CompanyName', 'PreviousName_7.CONDATE', 'PreviousName_7.CompanyName', 'PreviousName_8.CONDATE', 'PreviousName_8.CompanyName', 'PreviousName_9.CONDATE', 'PreviousName_9.CompanyName', 'PreviousName_10.CONDATE', 'PreviousName_10.CompanyName']
🗑️ Dropped 'Returns.LastMadeUpDate' due to >80% missing values.


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/957987104.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna('Unknown', inplace=True)
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/957987104.py:42: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipy

🧹 Dropped rows with missing values in critical columns: ['Returns.NextDueDate', 'IncorporationDate', 'RegAddress.AddressLine2', 'Accounts.LastMadeUpDate', 'ConfStmtLastMadeUpDate', 'Accounts.NextDueDate', 'ConfStmtNextDueDate']


In [11]:
# Check if any missing values exist
any_missing = df_clean.isnull().values.any()
print("Are there any missing values left?", any_missing)

# Show detailed missing values per column (only if any exist)
if any_missing:
    missing_percent = df_clean.isnull().mean() * 100
    print("\nMissing values percentage per column:")
    print(missing_percent[missing_percent > 0].sort_values(ascending=False))
else:
    print("✅ All missing values have been handled.")

# Save the cleaned DataFrame to a CSV file
df_clean.to_csv("Cleaned House data.csv", index=False)
print("Cleaned data saved as 'Cleaned House data.csv'")



Are there any missing values left? False
✅ All missing values have been handled.
Cleaned data saved as 'Cleaned House data.csv'


In [12]:
import pandas as pd
import json
import os

# Define base path where your JSON files are stored
base_path = '/Users/vaish2205/Downloads/'  # Update this path if needed

table_files = {
    'charity': 'publicextract.charity.json',
    'ar_parta': 'publicextract.charity_annual_return_parta.json',
    'ar_partb': 'publicextract.charity_annual_return_partb.json',
    'annual_return': 'publicextract.charity_annual_return_history.json',
    'classification': 'publicextract.charity_classification.json',
    'policy': 'publicextract.charity_policy.json',
    'trustee': 'publicextract.charity_trustee.json',
    'area': 'publicextract.charity_area_of_operation.json',
    
    # Optional:
    'event_history': 'publicextract.charity_event_history.json',
    'governing_document': 'publicextract.charity_governing_document.json',
    'other_names': 'publicextract.charity_other_names.json',
    'other_regulators': 'publicextract.charity_other_regulators.json',
    'published_report': 'publicextract.charity_published_report.json',
}


# Function to load JSON file into a DataFrame
def load_json_df(filepath):
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    return pd.DataFrame(data)

# Load all data into a dictionary of DataFrames
datasets = {}
for key, filename in table_files.items():
    full_path = os.path.join(base_path, filename)
    print(f"Loading {filename}...")
    datasets[key] = load_json_df(full_path)

# Example: access the charity DataFrame
charity = datasets['charity']
ar_parta = datasets['ar_parta']
ar_partb = datasets['ar_partb']
annual_return = datasets['annual_return']
classification = datasets['classification']
policy = datasets['policy']
trustee = datasets['trustee']
area = datasets['area']

# Optional tables (included in your directory)
event_history = datasets['event_history']
governing_document = datasets['governing_document']
other_names = datasets['other_names']
other_regulators = datasets['other_regulators']
published_report = datasets['published_report']


Loading publicextract.charity.json...
Loading publicextract.charity_annual_return_parta.json...
Loading publicextract.charity_annual_return_partb.json...
Loading publicextract.charity_annual_return_history.json...
Loading publicextract.charity_classification.json...
Loading publicextract.charity_policy.json...
Loading publicextract.charity_trustee.json...
Loading publicextract.charity_area_of_operation.json...
Loading publicextract.charity_event_history.json...
Loading publicextract.charity_governing_document.json...
Loading publicextract.charity_other_names.json...
Loading publicextract.charity_other_regulators.json...
Loading publicextract.charity_published_report.json...


In [15]:
# Start with charity as the base
merged = charity.copy()

# ---- Financial Features ----
# Merge part A financials (gov grants, contracts, etc.)
merged = merged.merge(ar_parta, on='organisation_number', how='left')

# Merge part B financials (reserves, assets, liabilities, etc.)
merged = merged.merge(ar_partb, on='organisation_number', how='left')

# ---- Annual Returns (Optional): Keep only most recent or aggregate stats later ----
# merged = merged.merge(annual_return, on='organisation_number', how='left')

# ---- Governance ----
# Count of governance policies
policy_count = policy.groupby('organisation_number').size().reset_index(name='policy_count')
merged = merged.merge(policy_count, on='organisation_number', how='left')

# Trustee count
trustee_count = trustee.groupby('organisation_number').size().reset_index(name='trustee_count')
merged = merged.merge(trustee_count, on='organisation_number', how='left')

# ---- Classification ----
classification_count = classification.groupby('organisation_number').size().reset_index(name='classification_count')
merged = merged.merge(classification_count, on='organisation_number', how='left')

# ---- Area of Operation ----
area_count = area.groupby('organisation_number').size().reset_index(name='area_coverage_count')
merged = merged.merge(area_count, on='organisation_number', how='left')


In [17]:
import pandas as pd

# Convert to datetime
merged['date_of_registration'] = pd.to_datetime(merged['date_of_registration'], errors='coerce')
merged['date_of_removal'] = pd.to_datetime(merged['date_of_removal'], errors='coerce')

# Define target: survival status
merged['is_surviving'] = merged['date_of_removal'].isnull().astype(int)

# Define end date (removal date or today)
merged['end_date'] = merged['date_of_removal'].fillna(pd.Timestamp.today())

# Duration in days
merged['survival_days'] = (merged['end_date'] - merged['date_of_registration']).dt.days

# Right-censoring flag
merged['censored'] = merged['date_of_removal'].isnull().astype(int)


In [19]:
import numpy as np  # Ensure this is imported


print(" Merged Dataset Overview")
print(f" Rows: {merged.shape[0]}, Columns: {merged.shape[1]}")
print()


print(" Column Data Types:")
print(merged.dtypes)
print()

print(" Missing Values (All Columns with Missing Data):")
missing = merged.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
display(missing)
print()


duplicates = merged.duplicated().sum()
print(f" Duplicate Rows: {duplicates}")
print()

#  Sample rows
print(" First 5 Sample Rows:")
display(merged.head(5))


print(" Summary Statistics (Numerical Columns):")
display(merged.describe(include=[np.number]))


print(" Summary Statistics (Categorical Columns):")
display(merged.describe(include=['object']))


 Merged Dataset Overview
 Rows: 1121334, Columns: 140

 Column Data Types:
date_of_extract_x                      object
organisation_number                     int64
registered_charity_number_x             int64
linked_charity_number                   int64
charity_name                           object
                                    ...      
area_coverage_count                   float64
is_surviving                            int64
end_date                       datetime64[ns]
survival_days                           int64
censored                                int64
Length: 140, dtype: object

 Missing Values (All Columns with Missing Data):


charity_is_cdf_or_cif              1121222
date_cio_dissolution_notice        1120675
count_salary_band_450001_500000    1106887
count_salary_band_over_500000      1106859
count_salary_band_400001_450000    1106765
                                    ...   
area_coverage_count                 120667
cio_is_dissolved                     40208
charity_is_cio                       40208
charity_previously_excepted          40208
charity_reporting_status             40208
Length: 127, dtype: int64


 Duplicate Rows: 0

 First 5 Sample Rows:


,date_of_extract_x,organisation_number,registered_charity_number_x,linked_charity_number,charity_name,charity_type,charity_registration_status,date_of_registration,date_of_removal,charity_reporting_status,...,charity_only_accounts,consolidated_accounts,policy_count,trustee_count,classification_count,area_coverage_count,is_surviving,end_date,survival_days,censored
0,2025-06-14T00:00:00,1,200027,1,POTTERNE MISSION ROOM AND TRUST,None,Removed,1962-05-17,2014-04-16,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2014-04-16 00:00:00.000000,18962,0
1,2025-06-14T00:00:00,2,200027,2,HITCHAM FREE CHURCH,None,Registered,1962-05-17,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2025-06-22 23:09:28.385614,23047,1
2,2025-06-14T00:00:00,3,200028,1,TOWN LANDS CHARITY FOR THE POOR,None,Removed,1961-10-19,1997-09-17,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-09-17 00:00:00.000000,13117,0
3,2025-06-14T00:00:00,4,200028,2,TOWN LANDS CHARITY FOR THE CHURCH,None,Removed,1961-10-19,1997-09-17,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1997-09-17 00:00:00.000000,13117,0
4,2025-06-14T00:00:00,5,200034,1,CLOPHILL RELIEF IN NEED CHARITY,None,Registered,1972-07-19,NaT,None,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2025-06-22 23:09:28.385614,19331,1


 Summary Statistics (Numerical Columns):


,organisation_number,registered_charity_number_x,linked_charity_number,latest_income,latest_expenditure,registered_charity_number_y,fin_period_order_number_x,total_gross_income,total_gross_expenditure,count_govt_contracts,...,funds_restricted,funds_total,count_employees,policy_count,trustee_count,classification_count,area_coverage_count,is_surviving,survival_days,censored
count,1.121334e+06,1.121334e+06,1.121334e+06,9.774360e+05,9.774360e+05,8.995810e+05,899581.000000,8.995810e+05,8.995810e+05,314238.000000,...,3.621150e+05,3.621150e+05,362125.000000,799991.000000,775103.000000,993084.000000,1.000667e+06,1.121334e+06,1.121334e+06,1.121334e+06
mean,2.703353e+06,8.629477e+05,5.089795e-01,2.536766e+06,2.561708e+06,9.188379e+05,2.919972,2.496166e+06,2.441804e+06,207.765442,...,1.956247e+06,1.766706e+07,84.154692,11.100634,6.755012,7.205451,2.626625e+00,7.047508e-01,9.239399e+03,7.047508e-01
std,2.068405e+06,3.752115e+05,6.223540e+00,1.958927e+07,2.136324e+07,3.487675e+05,1.412981,1.925491e+07,2.018976e+07,10930.878923,...,2.237963e+07,2.933743e+08,662.246473,5.738670,4.977302,3.994584,9.083729e+00,4.561549e-01,6.225326e+03,4.561549e-01
min,1.000000e+00,2.000000e+05,0.000000e+00,-7.334000e+04,0.000000e+00,2.000010e+05,1.000000,0.000000e+00,-5.421490e+05,0.000000,...,-5.497400e+07,-1.983171e+08,0.000000,1.000000,1.000000,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.285710e+05,5.034280e+05,0.000000e+00,1.456700e+04,1.466975e+04,8.005230e+05,2.000000,2.261900e+04,2.043000e+04,0.000000,...,0.000000e+00,4.655770e+05,7.000000,6.000000,4.000000,4.000000,1.000000e+00,0.000000e+00,4.172000e+03,0.000000e+00
50%,3.954353e+06,1.074008e+06,0.000000e+00,9.577500e+04,9.725000e+04,1.095815e+06,3.000000,1.252030e+05,1.187390e+05,0.000000,...,5.670400e+04,1.451870e+06,22.000000,12.000000,6.000000,6.000000,1.000000e+00,1.000000e+00,7.860000e+03,1.000000e+00
75%,5.014263e+06,1.140913e+06,0.000000e+00,9.016692e+05,9.068120e+05,1.149169e+06,4.000000,9.474120e+05,9.087150e+05,1.000000,...,3.830000e+05,6.160506e+06,56.000000,16.000000,9.000000,9.000000,2.000000e+00,1.000000e+00,1.266000e+04,1.000000e+00
max,5.266690e+06,1.213631e+06,2.960000e+02,1.358100e+09,1.660835e+09,1.210106e+06,5.000000,1.488506e+09,1.660835e+09,987361.000000,...,1.848467e+09,3.626056e+10,146000.000000,23.000000,102.000000,34.000000,2.760000e+02,1.000000e+00,2.366200e+04,1.000000e+00


 Summary Statistics (Categorical Columns):


,date_of_extract_x,charity_name,charity_type,charity_registration_status,charity_reporting_status,latest_acc_fin_period_start_date,latest_acc_fin_period_end_date,charity_contact_address1,charity_contact_address2,charity_contact_address3,...,employees_salary_over_60k,date_of_extract,latest_fin_period_submitted_ind_y,ar_cycle_reference_y,fin_period_start_date_y,fin_period_end_date_y,ar_due_date_y,ar_received_date_y,charity_only_accounts,consolidated_accounts
count,1121334,1121334,964586,1121334,1081126,977436,977436,775638,773810,623314,...,663030,362129,362129,362129,362129,362129,362129,362129,275901,86135
unique,1,372895,5,2,7,4284,4090,136599,74368,31733,...,2,1,2,19,1050,684,665,4782,1,2
top,2025-06-14T00:00:00,VILLAGE HALL,Charitable company,Registered,Submission Received,2023-04-01T00:00:00,2024-03-31T00:00:00,Village Hall,LONDON,LONDON,...,False,2025-06-14T00:00:00,False,AR23,2023-04-01T00:00:00,2024-03-31T00:00:00,2025-01-31T00:00:00,2024-01-31T00:00:00,True,True
freq,1121334,459,352639,790261,680306,268097,269795,1068,37824,51363,...,522985,362129,283691,67136,30439,30601,30601,3560,275901,86095


In [21]:
# Total number of rows
total_rows = len(merged)

# Get missing values as percentage
missing_percent = merged.isnull().mean() * 100

# Filter only columns with at least one missing value
missing_percent = missing_percent[missing_percent > 0]

# Sort by highest percentage
missing_percent = missing_percent.sort_values(ascending=False)

# Display the result
print("📊 Missing Values Percentage by Column (Descending):")
for col, pct in missing_percent.items():
    print(f"{col:<40} : {pct:.2f}%")


📊 Missing Values Percentage by Column (Descending):
charity_is_cdf_or_cif                    : 99.99%
date_cio_dissolution_notice              : 99.94%
count_salary_band_450001_500000          : 98.71%
count_salary_band_over_500000            : 98.71%
count_salary_band_400001_450000          : 98.70%
count_salary_band_350001_400000          : 98.70%
count_salary_band_300001_350000          : 98.68%
count_salary_band_250001_300000          : 98.61%
charity_agreement_professional_fundraiser : 98.51%
count_salary_band_200001_250000          : 98.43%
count_salary_band_140001_150000          : 98.20%
count_salary_band_130001_140000          : 98.01%
count_salary_band_150001_200000          : 97.84%
count_salary_band_120001_130000          : 97.78%
count_salary_band_110001_120000          : 97.48%
count_salary_band_100001_110000          : 96.81%
count_salary_band_90001_100000           : 95.96%
count_salary_band_80001_90000            : 94.91%
charity_agreement_commerical_participator : 94.

In [23]:
import pandas as pd
import numpy as np

# -------------------------------
# 1. Drop Columns with >80% Missing Values
# -------------------------------
threshold = 0.80
high_missing_cols = merged.columns[merged.isnull().mean() > threshold]
merged.drop(columns=high_missing_cols, inplace=True)
print(f"\u274C Dropped {len(high_missing_cols)} columns with >80% missing values.")

# -------------------------------
# 2. Drop Fully Missing Columns
# -------------------------------
fully_missing_cols = merged.columns[merged.isnull().mean() == 1.0]
merged.drop(columns=fully_missing_cols, inplace=True)
print(f"🧹 Dropped {len(fully_missing_cols)} columns with 100% missing values.")

# -------------------------------
# 3. Drop Rows with Critical Missing Values
# -------------------------------
merged = merged[merged['organisation_number'].notnull()]
merged = merged[merged['date_of_registration'].notnull()]

# -------------------------------
# 4. Fill Financial Columns and Add Missing Flags
# -------------------------------
financial_cols = [
    'reserves',
    'total_gross_income',
    'total_gross_expenditure',
    'income_from_government_grants',
    'count_govt_contracts'
]

for col in financial_cols:
    if col in merged.columns:
        merged[f'missing_{col}'] = merged[col].isnull().astype(int)
        merged[col] = merged[col].fillna(0)
    else:
        print(f" Column '{col}' not found — skipped.")

# -------------------------------
# 5. Fill Operational & Governance Count Columns
# -------------------------------
for col in ['trustee_count', 'policy_count', 'classification_count', 'area_coverage_count']:
    if col in merged.columns:
        merged[col] = merged[col].fillna(0)

# -------------------------------
# 6. Fill Binary Indicators
# -------------------------------
binary_funding_cols = [
    'charity_receives_govt_funding_grants',
    'charity_receives_govt_funding_contracts',
    'charity_raises_funds_from_public',
    'charity_gift_aid',
    'charity_has_trading_subsidiary',
    'charity_has_land'
]

for col in binary_funding_cols:
    if col in merged.columns:
        merged[col] = merged[col].fillna(0).astype(int)

# -------------------------------
# 7. Fill Categorical Columns with 'Unknown'
# -------------------------------
cat_cols = [
    'charity_type', 'charity_reporting_status', 'charity_reporting_type',
    'charity_is_cio', 'cio_is_dissolved'
]

for col in cat_cols:
    if col in merged.columns:
        if merged[col].dtype == 'O':
            merged[col] = merged[col].fillna('Unknown')
        else:
            merged[col] = merged[col].fillna(0).astype(int)

# -------------------------------
# 8. Fill Contact Fields
# -------------------------------
if 'charity_contact_postcode' in merged.columns:
    merged['charity_contact_postcode'] = merged['charity_contact_postcode'].fillna('Unknown')

# -------------------------------
# 9. Handle Date Columns
# -------------------------------
date_cols = [col for col in merged.columns if 'date' in col.lower() or 'fin_period' in col.lower()]
for col in date_cols:
    if col in merged.columns:
        merged[col] = pd.to_datetime(merged[col], errors='coerce')

# -------------------------------
# 10. Fill Remaining Missing Numeric Columns with 0 and Flag
# -------------------------------
numeric_missing_cols = merged.select_dtypes(include=['float64', 'int64']).columns
numeric_missing_cols = [col for col in numeric_missing_cols if merged[col].isnull().sum() > 0]

for col in numeric_missing_cols:
    merged[f'missing_{col}'] = merged[col].isnull().astype(int)
    merged[col] = merged[col].fillna(0)

# -------------------------------
# 11. Fill Remaining Categorical Columns with 'Unknown'
# -------------------------------
categorical_missing_cols = merged.select_dtypes(include='object').columns
categorical_missing_cols = [col for col in categorical_missing_cols if merged[col].isnull().sum() > 0]

for col in categorical_missing_cols:
    merged[col] = merged[col].fillna('Unknown')

# -------------------------------
# 12. Final Missing Value Report
# -------------------------------
remaining = merged.isnull().sum()
remaining = remaining[remaining > 0]

if not remaining.empty:
    print("\nRemaining columns with missing values:")
    display(pd.DataFrame({
        'Missing Count': remaining,
        'Missing %': (remaining / len(merged)) * 100
    }).sort_values(by='Missing Count', ascending=False))
else:
    print("\n🎉 All missing values have been handled. Dataset is clean and ready.")

# Final Dataset Shape
print(f"\nFinal dataset shape: {merged.shape[0]} rows × {merged.shape[1]} columns")


❌ Dropped 25 columns with >80% missing values.
🧹 Dropped 0 columns with 100% missing values.


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/2520440958.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'missing_{col}'] = merged[col].isnull().astype(int)
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/2520440958.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'missing_{col}'] = merged[col].isnull().astype(int)
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/2520440958.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 


Remaining columns with missing values:


,Missing Count,Missing %
latest_fin_period_submitted_ind_x,1121334,100.000000
latest_fin_period_submitted_ind_y,1121334,100.000000
date_of_removal,790261,70.475077
date_of_extract,759205,67.705519
ar_due_date_y,759205,67.705519
fin_period_end_date_y,759205,67.705519
fin_period_start_date_y,759205,67.705519
fin_period_order_number_y,759205,67.705519
ar_received_date_y,759205,67.705519
ar_received_date_x,221753,19.775821



Final dataset shape: 1121334 rows × 163 columns


In [25]:
import pandas as pd

# Identify date columns
date_cols = [col for col in merged.columns if 'date' in col.lower() or 'fin_period' in col.lower()]

# Convert to datetime (if not already)
for col in date_cols:
    merged[col] = pd.to_datetime(merged[col], errors='coerce')

# For each date column, add missing flag and fill missing values with a placeholder date
for col in date_cols:
    if merged[col].isnull().sum() > 0:
        merged[f'missing_{col}'] = merged[col].isnull().astype(int)
        # Fill missing dates with a placeholder (e.g., 1900-01-01)
        merged[col] = merged[col].fillna(pd.Timestamp('1900-01-01'))



/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/1795682447.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'missing_{col}'] = merged[col].isnull().astype(int)
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/1795682447.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged[f'missing_{col}'] = merged[col].isnull().astype(int)
/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/1795682447.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [27]:
# Check if any missing values remain
remaining = merged.isnull().sum()
remaining = remaining[remaining > 0]

if not remaining.empty:
    print("\nRemaining columns with missing values:")
    display(pd.DataFrame({
        'Missing Count': remaining,
        'Missing %': (remaining / len(merged)) * 100
    }).sort_values(by='Missing Count', ascending=False))
else:
    print("\n🎉 All missing values handled")

# Save the cleaned DataFrame to a CSV file
merged.to_csv("cleaned gov charity data.csv", index=False)
print("Cleaned data saved as 'cleaned gov charity data.csv'")



🎉 All missing values handled
Cleaned data saved as 'cleaned gov charity data.csv'


In [29]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv("cleaned gov charity data.csv")

# Show number of rows and columns
print(f"Dataset shape: {df.shape} (rows, columns)\n")

# Show info about columns and data types
print("Data types and non-null counts:")
print(df.info(), "\n")

# Show missing values count and percentage per column
missing_count = df.isnull().sum()
missing_percent = (missing_count / len(df)) * 100

missing_data = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_percent
})

print("Missing values per column:")
print(missing_data[missing_data['Missing Count'] > 0].sort_values(by='Missing %', ascending=False), "\n")

# Show basic statistics for numeric columns
print("Basic statistics summary:")
print(df.describe(), "\n")

# Preview first 5 rows
print("First 5 rows of the dataset:")
print(df.head())


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/1384448920.py:4: DtypeWarning: Columns (25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned gov charity data.csv")


Dataset shape: (1121334, 180) (rows, columns)

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121334 entries, 0 to 1121333
Columns: 180 entries, date_of_extract_x to missing_ar_received_date_y
dtypes: bool(2), float64(52), int64(77), object(49)
memory usage: 1.5+ GB
None 

Missing values per column:
                          Missing Count  Missing %
charity_contact_web                8347   0.744381
charity_contact_email              6995   0.623811
charity_activities                  450   0.040131
charity_contact_address4             63   0.005618
charity_contact_address3             25   0.002229
charity_contact_phone                19   0.001694
charity_contact_postcode              5   0.000446
charity_contact_address1              2   0.000178
charity_contact_address2              2   0.000178 

Basic statistics summary:
       organisation_number  registered_charity_number_x  \
count         1.121334e+06                 1.121334e+06   
mean   

In [31]:
import pandas as pd

# Load the cleaned house data
df = pd.read_csv("Cleaned House data.csv")

# Show dataset shape (rows, columns)
print(f"Dataset shape: {df.shape}\n")

# Show info about columns and data types
print("Data types and non-null counts:")
print(df.info(), "\n")

# Calculate missing values count and percentage per column
missing_count = df.isnull().sum()
missing_percent = (missing_count / len(df)) * 100

missing_data = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_percent
})

print("Missing values per column (if any):")
print(missing_data[missing_data['Missing Count'] > 0].sort_values(by='Missing %', ascending=False), "\n")

# Show basic statistics summary for numeric columns
print("Basic statistics summary:")
print(df.describe(), "\n")

# Preview first 5 rows of data
print("First 5 rows of the dataset:")
print(df.head())


Dataset shape: (1265591, 28)

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265591 entries, 0 to 1265590
Data columns (total 28 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   CompanyName                         1265591 non-null  object 
 1   CompanyNumber                       1265591 non-null  object 
 2   RegAddress.AddressLine1             1265591 non-null  object 
 3   RegAddress.AddressLine2             1265591 non-null  object 
 4   RegAddress.PostTown                 1265591 non-null  object 
 5   RegAddress.County                   1265591 non-null  object 
 6   RegAddress.Country                  1265591 non-null  object 
 7   RegAddress.PostCode                 1265591 non-null  object 
 8   CompanyCategory                     1265591 non-null  object 
 9   CompanyStatus                       1265591 non-null  object 
 10  CountryOfOrigin 

In [33]:
import pandas as pd

# Load the cleaned datasets
charity_df = pd.read_csv("cleaned gov charity data.csv")
house_df = pd.read_csv("Cleaned House data.csv")

# Show all column names neatly
pd.set_option('display.max_columns', None)

print("Charity Dataset Columns:\n")
for col in charity_df.columns:
    print(col)

print("\nHouse Dataset Columns:\n")
for col in house_df.columns:
    print(col)


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/2200691658.py:4: DtypeWarning: Columns (25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  charity_df = pd.read_csv("cleaned gov charity data.csv")


Charity Dataset Columns:

date_of_extract_x
organisation_number
registered_charity_number_x
linked_charity_number
charity_name
charity_type
charity_registration_status
date_of_registration
date_of_removal
charity_reporting_status
latest_acc_fin_period_start_date
latest_acc_fin_period_end_date
latest_income
latest_expenditure
charity_contact_address1
charity_contact_address2
charity_contact_address3
charity_contact_address4
charity_contact_postcode
charity_contact_phone
charity_contact_email
charity_contact_web
charity_company_registration_number
charity_insolvent
charity_in_administration
charity_previously_excepted
charity_is_cio
cio_is_dissolved
charity_activities
charity_gift_aid
charity_has_land
date_of_extract_y
registered_charity_number_y
latest_fin_period_submitted_ind_x
fin_period_order_number_x
ar_cycle_reference_x
fin_period_start_date_x
fin_period_end_date_x
ar_due_date_x
ar_received_date_x
total_gross_income
total_gross_expenditure
charity_raises_funds_from_public
charity_p

In [35]:
import pandas as pd

# Load datasets
charity_df = pd.read_csv("cleaned gov charity data.csv")
house_df = pd.read_csv("Cleaned House data.csv")

# Merge on company registration number
merged_df = pd.merge(
    charity_df,
    house_df,
    how='inner',  # change to 'left' or 'outer' if needed
    left_on='charity_company_registration_number',
    right_on='CompanyNumber'
)

# Display basic info about merged data
print("Merged Final dataset shape:", merged_df.shape)
print("\nSample rows:\n", merged_df.head())


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/491977636.py:4: DtypeWarning: Columns (25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  charity_df = pd.read_csv("cleaned gov charity data.csv")


Merged Final dataset shape: (71385, 208)

Sample rows:
   date_of_extract_x  organisation_number  registered_charity_number_x  \
0        2025-06-14               200124                       200124   
1        2025-06-14               200124                       200124   
2        2025-06-14               200124                       200124   
3        2025-06-14               200124                       200124   
4        2025-06-14               200843                       200843   

   linked_charity_number                                       charity_name  \
0                      0                     ABINGER CONSOLIDATED CHARITIES   
1                      0                     ABINGER CONSOLIDATED CHARITIES   
2                      0                     ABINGER CONSOLIDATED CHARITIES   
3                      0                     ABINGER CONSOLIDATED CHARITIES   
4                      0  THE NORTH STAFFORDSHIRE MEDICAL INSTITUTE LIMITED   

         charity_type charity_

In [37]:
# Check for missing values in the merged dataset
missing_values = merged_df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Display top missing columns
print("\nColumns with missing values:")
print(missing_values)  # adjust number as needed


Columns with missing values:
charity_contact_email       452
charity_contact_web         291
charity_activities           12
charity_contact_address4      4
dtype: int64


In [39]:
merged_df['charity_contact_email'] = merged_df['charity_contact_email'].fillna('Not Provided')
merged_df['charity_contact_web'] = merged_df['charity_contact_web'].fillna('Not Provided')
merged_df['charity_activities'] = merged_df['charity_activities'].fillna('Unknown')
merged_df['charity_contact_address4'] = merged_df['charity_contact_address4'].fillna('')


In [41]:
# Check for missing values in the merged dataset
missing_values = merged_df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

# Display top missing columns
print("\nColumns with missing values:")
print(missing_values)  # adjust number as needed


Columns with missing values:
Series([], dtype: int64)


In [43]:
import pandas as pd

# Load datasets
charity_df = pd.read_csv("cleaned gov charity data.csv")
house_df = pd.read_csv("Cleaned House data.csv")

# Merge on company registration number
merged_df = pd.merge(
    charity_df,
    house_df,
    how='inner',  # change to 'left' or 'outer' if needed
    left_on='charity_company_registration_number',
    right_on='CompanyNumber'
)

# Display basic info about merged data
print("Merged Final dataset shape:", merged_df.shape)
print("\nSample rows:\n", merged_df.head())


/var/folders/8w/qkgpp5cx5gbd6v2vcgffvnp80000gn/T/ipykernel_3302/491977636.py:4: DtypeWarning: Columns (25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  charity_df = pd.read_csv("cleaned gov charity data.csv")


Merged Final dataset shape: (71385, 208)

Sample rows:
   date_of_extract_x  organisation_number  registered_charity_number_x  \
0        2025-06-14               200124                       200124   
1        2025-06-14               200124                       200124   
2        2025-06-14               200124                       200124   
3        2025-06-14               200124                       200124   
4        2025-06-14               200843                       200843   

   linked_charity_number                                       charity_name  \
0                      0                     ABINGER CONSOLIDATED CHARITIES   
1                      0                     ABINGER CONSOLIDATED CHARITIES   
2                      0                     ABINGER CONSOLIDATED CHARITIES   
3                      0                     ABINGER CONSOLIDATED CHARITIES   
4                      0  THE NORTH STAFFORDSHIRE MEDICAL INSTITUTE LIMITED   

         charity_type charity_

In [45]:
print(" Total columns:", len(merged_df.columns))
print("\n Column Names:\n")

for i, col in enumerate(merged_df.columns, start=1):
    print(f"{i}. {col}")


 Total columns: 208

 Column Names:

1. date_of_extract_x
2. organisation_number
3. registered_charity_number_x
4. linked_charity_number
5. charity_name
6. charity_type
7. charity_registration_status
8. date_of_registration
9. date_of_removal
10. charity_reporting_status
11. latest_acc_fin_period_start_date
12. latest_acc_fin_period_end_date
13. latest_income
14. latest_expenditure
15. charity_contact_address1
16. charity_contact_address2
17. charity_contact_address3
18. charity_contact_address4
19. charity_contact_postcode
20. charity_contact_phone
21. charity_contact_email
22. charity_contact_web
23. charity_company_registration_number
24. charity_insolvent
25. charity_in_administration
26. charity_previously_excepted
27. charity_is_cio
28. cio_is_dissolved
29. charity_activities
30. charity_gift_aid
31. charity_has_land
32. date_of_extract_y
33. registered_charity_number_y
34. latest_fin_period_submitted_ind_x
35. fin_period_order_number_x
36. ar_cycle_reference_x
37. fin_period_sta

In [49]:
import pandas as pd
import numpy as np

# Copy the merged dataset
df = merged_df.copy()  # Make sure `merged_df` contains both charity and socioeconomic data


#  1. Survival Duration Features

df['date_of_registration'] = pd.to_datetime(df.get('date_of_registration'), errors='coerce')
df['IncorporationDate'] = pd.to_datetime(df.get('IncorporationDate'), errors='coerce')
df['start_date'] = df['date_of_registration'].combine_first(df['IncorporationDate'])

df['date_of_removal'] = pd.to_datetime(df.get('date_of_removal'), errors='coerce')
df['DissolutionDate'] = pd.to_datetime(df.get('DissolutionDate'), errors='coerce')
study_end = pd.to_datetime("2024-12-31")
df['end_date'] = df['date_of_removal'].combine_first(df['DissolutionDate']).fillna(study_end)

df['survival_days'] = (df['end_date'] - df['start_date']).dt.days
df['survival_years'] = df['survival_days'] / 365.25
df['is_surviving'] = (df['end_date'] == study_end).astype(int)


#  2. Financial Health Features

df['reserves_to_income'] = np.where(
    df.get('total_gross_income', 0) > 0,
    df.get('reserves', 0) / df.get('total_gross_income', 1),
    0
)

df['gov_grant_ratio'] = np.where(
    df.get('total_gross_income', 0) > 0,
    df.get('income_from_government_grants', 0) / df.get('total_gross_income', 1),
    0
)

income_sources = [
    'income_donations_and_legacies', 'income_other_trading_activities',
    'income_charitable_activities', 'income_investments', 'income_endowments'
]
existing_sources = [col for col in income_sources if col in df.columns]
df['income_diversity_score'] = df[existing_sources].gt(0).sum(axis=1) if existing_sources else 0


#  3. Governance & Operational Complexity

for col in ['trustee_count', 'policy_count', 'classification_count', 'area_coverage_count']:
    if col in df.columns:
        df[col] = df[col].fillna(0)

df['governance_score'] = df.get('trustee_count', 0) + df.get('policy_count', 0)
df['operational_complexity'] = df.get('classification_count', 0) + df.get('area_coverage_count', 0)


#  4. Age & Cohort Features

df['charity_age_years'] = df['survival_years']
df['is_new_charity'] = (df['charity_age_years'] < 5).astype(int)

df['registration_year'] = df['start_date'].dt.year
df['cohort_group'] = pd.cut(
    df['registration_year'],
    bins=[1900, 2000, 2010, 2015, 2020, 2025],
    labels=['<2000', '2000–2010', '2010–2015', '2015–2020', '2020–2024']
)


#  5. Region and Sector

df['postcode'] = df.get('charity_contact_postcode', '').combine_first(df.get('RegAddress.PostCode', '')).fillna('Unknown')
df['region_code'] = df['postcode'].str.extract(r'([A-Z]{1,2})')

df['sector'] = df.get('SICCode.SicText_1', df.get('charity_type', 'Unknown')).fillna('Unknown')


#  6. Socioeconomic Indicators (from housing data)

# Example columns from house dataset if already merged
# Update these based on actual column names in merged_df
socio_cols = ['avg_household_income', 'unemployment_rate', 'region_deprivation_score']
existing_socio_cols = [col for col in socio_cols if col in df.columns]

# Optional: Fill missing socioeconomic values (if needed)
df[existing_socio_cols] = df[existing_socio_cols].fillna(df[existing_socio_cols].median())


#  7. Final Feature Selection

selected_features = [
    'organisation_number', 'is_surviving', 'survival_years',
    'reserves_to_income', 'gov_grant_ratio', 'income_diversity_score',
    'governance_score', 'operational_complexity', 'charity_age_years',
    'is_new_charity', 'registration_year', 'cohort_group', 'sector',
    'region_code'
] + existing_socio_cols  # Add socioeconomic indicators if available

final_features_df = df[selected_features].copy()

print(" Feature engineering complete. Here's a preview:")
print(final_features_df.head())


 Feature engineering complete. Here's a preview:
   organisation_number  is_surviving  survival_years  reserves_to_income  \
0               200124             0      -61.486653                 0.0   
1               200124             0      -61.486653                 0.0   
2               200124             0      -61.486653                 0.0   
3               200124             0      -61.486653                 0.0   
4               200843             0      -67.830253                 0.0   

   gov_grant_ratio  income_diversity_score  governance_score  \
0              0.0                       0              10.0   
1              0.0                       0              10.0   
2              0.0                       0              10.0   
3              0.0                       0              10.0   
4              0.0                       0              29.0   

   operational_complexity  charity_age_years  is_new_charity  \
0                     9.0         -61.486653 